
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Autoenkodery</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Autoenkoder
ma za zadanie 
$$D(E(x))=x$$
1. jeśli nauczy się $D(E(x))=x$ dla wszystkich elementów $x$, to taki model niewiele nam daje, w rzeczywistości minimalizują koszt $$L(x,D(E(x)))$$
zapamiętując wszystkie przykłady
  * dla liniowego dekodera i kwadratowej funkcji kosztu mamy model identyczny z PCA
    * generuje tą samą podprzestrzeń
  * nieliniowy dekoder powinien dawać modele silniejsze
2. autoenkodery
  * mają wyszukiwać informacje __istotne__
  * są zwykle w jakiś sposób ograniczone
  * mogą pozwalać na redukcję wymiarowości i uczenie cech (podstawowe zastosowanie)
  * obecnie najważniejsze stały się modele __generatywne__

## Ograniczanie enkoderów
1. niewielkie wymiary przestrzeni kodów czy modele dekoderów / enkoderów o małej pojemności nie pozwolą na nauczenie się istotnych informacji
2. duża przestrzeń latent pozwoli na proste kopiowanie
  * AE nie uczy się niczego istotnego
3. potrzebne ograniczenie oparte na złożoności problemu
  * dodatkowa funkcja kosztu __regularyzująca__ rozwiązanie

### Modele rzadkie
$$L(x,D(E(x)))+\Omega(h),$$
gdzie $h$ jest przestrzenią ukrytą (latent)
1. dla $x$ i $h$ mamy rozkład łączny
$$p(x,h)=p(h)p(x\mid h)$$
  gdzie $p()$ jest realizowane przez budowany model
  * ucząc model __maksymalizujemy__
  $$\log\,p(x,h)=\log\,p(h)+\log\,p(x\mid h)$$
  * składnik $\log\,p(h)$ opisuje ograniczenia na warstwę latent
  * może to być wymaganie, by aktywacje były __rzadkie__
  * przyjmując $$p(h_i)=\frac{\lambda}{2}\exp(-\lambda|h_i|)$$
  dostajemy
  $$\begin{align}
  \log p(h)&=\log\,\prod_i\frac{\lambda}{2}\exp(-\lambda|h_i|)\\
  &=\sum_i\log\left(\frac{\lambda}{2}\exp(-\lambda|h_i|)\right)\\
  &=\sum_i\left[-\lambda|h_i|+\log\frac{\lambda}{2}\right]\\
  &=\Omega(h)+const
  \end{align}$$
    * składnik $\sum_i\log\frac{\lambda}{2}$ nie jest istotny dla uczenia



5. Typowym użyciem jest uczenie cech dla późniejszego uczenia nadzorowanego
  * na wejściu reprezentacja dźwięku
  * w warstwie latent ograniczona warstwa

## Denoising AE
minimalizuje koszt
$$L(x, D(E(\tilde{x})),$$
gdzie $\tilde{x}$ jest __zaburzoną__ wartością $x$

1. Denoising AE ma za zadanie __usuwać__ zaburzenia $x$
2. koszt obliczany między wartością odzyskaną a __prawdziwą__
<img src="../nn_figures/dae.pdf" width="88%"> [Bengio et al.]
  * oryginalny przykład $x$ jest zaburzany procesem $C(\tilde{x}\mid x)$
  * $D(E(\tilde{x}))$ aproksymuje wartość oczekiwaną $$E_{x,\tilde{x}}[x\mid \tilde{x}]$$
  gdzie $\tilde{x}$ pochodzi z rozkładu danych zmodyfikowanych zaburzeniem $$p_{Data}(x)C(\tilde{x}\mid x)$$
  * AE minimalizuje koszt $\|D(E(\tilde{x}))-x\|^2$
  * DAE uczy się pola wektorowego $D(E(x))-x$
  <img src="../nn_figures/dae-learn.pdf" width="86%"> [Bengio et al.]
    * gausowskie zaburzenie o średniej $\mu=x$ i macierzy kowariancji $\Sigma=\sigma^2I$ dla ustalonego $\sigma^2$
    * każda strzałka jest proporcjonalna do rekonstrukcji pomniejszonej o wejście
    * wskazuje na obszar o największej estymacji prawdopodobieństwia

## Rozpoznawanie mowy
<img src="../nn_figures/widmo.pdf" width="88%"> [Kowenzowski]
1. częstotliwości są obcinane do pasma mowy, część jest podbijana
2. sygnał dzielony na ramki o długości 25ms i przesunięciu 10 ms
3. przejście z domeny częstotliwosciowej do domeny czasowej (DFT)
4. zamiana na logarytmiczną skalę Mel Frequency Cepstral Coefficient MFCC
5. dodanie sygnału (wysokiej częstotliwości) tzw. pobudzenia (glottal)
6. co w sumie daje cechy różnicowe (delta features) podstawowy i drugiego stopnia

Algorytm wykorzystujący np. algorytm Viterbiego w modelu HMM albo sieć neuronowa pozwala na transformację na fonemy. Modele wykorzystują dodatkowo model językowy (NLP).

Dla 6-wartwowej sieci (DBN) model uzyskuje do 17% WER (Word Error Rate)
$$WER=100\frac{I+S+D}{A},$$
gdzie I -- insertions, S -- substitutions, D -- deletions, A -- all words

Wyniki są zależne także od wejścia: same MFCC, MFCC + delta, + sygnał pobudzenia)
### Dodanie cech bottleneck
1. transformacje jak wcześniej
2. budowa auto autoenkodera dla reprezentacji wejść 
  * na wejściu np. 11 kolejnych MFCC
  * warstwa ukryta (latent) o wielkości jednego MFCC
  * po nauczeniu pozostaje sam enkoder
3. model rozpoznający
  * głęboka sieć neuronowa jak dotychczas
  * cechy wejściowe MFCC (11 kolejnych ramek) podawane są do enkodera, który zwraca cechę __bottleneck__
  * cecha bottleneck dodawana do wejścia
  
Tak model rozpoznający ma WER poprawiony zwykle o co najmniej kilka procent

# Rozmaitości i autoenkodery
1. w ML __rozmaitość__ to zbiór punktów, które mogą być dobrze aproksymowane z wykorzystaniem niewielu stopni swobody (wymiarów)
  * wymiarowość może się zmieniać dla danej rozmaitości od punktu do punktu
  * każdy punkt ma swoje otoczenie z metryką
    * metryka może być __różna__ dla różnych obszarów
2. zwykle rozmaitości w problemach ML są __zanurzone__ w wysoko-wymiarowych przestrzeniach
3. __hipoteza rozmaitości__ mówi o tym, że problemy w ML są reprezentowane przez nisko-wymiarowe rozmaitości (albo ich zbiory) zanurzone w wysokowymiarowych przestrzeniach oryginalnych danych
  * nie zawsze jest prawdziwa
  * poprawnie zbudowane AE mają szansę znaleźć te rozmaitości
  <img src="../nn_figures/manifold.pdf" width="86%"> [Bengio et al.]
  * możliwą procedurą odtwarzania rozmaitości jest budowa grafu opartego na najbliższych sąsiadach
  * krawędzie mogą odpowiadać prostym przekształceniom
  <img src="../nn_figures/manifold2.pdf" width="86%"> [Bengio et al.]
  * okręgi reprezentują płaszczyzny styczne do rozmaitości w każdym punkcie
  * pozwala to na ich sklejenie
  * jeśli lokalne elementy są lokalnie płaskimi Gausianami, to otrzymujemy model jako mixture of Gaussians
  * w rozmaitości można znaleźć kierunki odpowiadające prostym cechom
  * to pozwala na proste operacje arytmetyczne na wektorach reprezentacji
    * AE w ogólności tworzą reprezentacje latent, które są pokawałkowane
    * wiele obszarów jest pustych
    * interpolacje w przestrzeni mogą nie odpowiadać obszarom o wysokiej gęstosci prawdopodobieństwa
  * potrzebne są dodatkowe ograniczenia, które będą budowały obszary spójne
    * modele generatywne, np. VAE, WAE, GAN, etc.